In [1]:
text=open('warpeace_input.txt','r').read()

In [2]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Flatten,BatchNormalization,Dropout 

In [3]:
text[:100]

'ï»¿"Well, Prince, so Genoa and Lucca are now just family estates of the\nBuonapartes. But I warn you,'

In [4]:
chars = list(set(text))
VOCAB_SIZE = len(chars)

In [5]:
char2idx = {u:i for i, u in enumerate(chars)}
idx2char = np.array(chars)
print(char2idx)

{'g': 0, 'B': 1, '-': 2, '=': 3, 'N': 4, 'C': 5, 'G': 6, 'h': 7, 'q': 8, '\xa0': 9, 'D': 10, 'J': 11, 'ª': 12, 'Ã': 13, '.': 14, '?': 15, 'u': 16, "'": 17, ';': 18, 'E': 19, '©': 20, 'k': 21, 'd': 22, 'm': 23, 'O': 24, ':': 25, 'T': 26, 'R': 27, 'y': 28, 'H': 29, '1': 30, '8': 31, 't': 32, '!': 33, 'r': 34, '9': 35, '3': 36, 'w': 37, 'V': 38, '\n': 39, 'Q': 40, 'c': 41, 'A': 42, '/': 43, 'b': 44, 'x': 45, 'U': 46, 'n': 47, 'f': 48, 's': 49, '¤': 50, 'X': 51, '»': 52, 'a': 53, 'l': 54, 'K': 55, '"': 56, ',': 57, '2': 58, ')': 59, 'W': 60, '5': 61, 'S': 62, '0': 63, ' ': 64, 'Z': 65, 'v': 66, 'ï': 67, '¿': 68, 'z': 69, 'j': 70, '(': 71, '7': 72, 'e': 73, 'F': 74, 'Y': 75, 'o': 76, 'p': 77, 'L': 78, '6': 79, 'I': 80, 'M': 81, 'i': 82, '*': 83, 'P': 84, '4': 85}


In [6]:
print(idx2char)

['g' 'B' '-' '=' 'N' 'C' 'G' 'h' 'q' '\xa0' 'D' 'J' 'ª' 'Ã' '.' '?' 'u'
 "'" ';' 'E' '©' 'k' 'd' 'm' 'O' ':' 'T' 'R' 'y' 'H' '1' '8' 't' '!' 'r'
 '9' '3' 'w' 'V' '\n' 'Q' 'c' 'A' '/' 'b' 'x' 'U' 'n' 'f' 's' '¤' 'X' '»'
 'a' 'l' 'K' '"' ',' '2' ')' 'W' '5' 'S' '0' ' ' 'Z' 'v' 'ï' '¿' 'z' 'j'
 '(' '7' 'e' 'F' 'Y' 'o' 'p' 'L' '6' 'I' 'M' 'i' '*' 'P' '4']


In [7]:
text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print(text_as_int)

[67 52 68 ... 16 49 14]


In [9]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'ï»¿"Well, Pri' ---- characters mapped to int ---- > [67 52 68 56 60 73 54 54 57 64 84 34 82]


In [10]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
no_samples = len(text)/seq_length

x=[]
y=[]

for z in range(int(len(text)/seq_length)):
    for i in range(seq_length):
        xe=[]
        for j in range(seq_length-i-1):
            xe.append(char2idx[' '])
        for j in range(i+1):
            xe.append(text_as_int[z*seq_length+j])
        xe=np.array(xe)

        x.append(xe)
        y.append(text_as_int[(z)*seq_length+i+1])
 

In [11]:
model=Sequential()
model.add(Embedding(VOCAB_SIZE,256,input_length=seq_length))
model.add(LSTM(200,return_sequences=True))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(200,activation='relu'))
model.add(Dense(VOCAB_SIZE,activation='softmax'))
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [12]:
temp_y=np.zeros(shape=(len(y),VOCAB_SIZE))
x=np.array(x)
print(temp_y.shape)
print(x.shape)
for i in range(len(y)):
    temp_y[i][y[i]]=1
y=temp_y

(3196200, 86)
(3196200, 100)


In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.2,test_size=0.02,random_state=42)

In [14]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(639240, 100)
(639240, 86)
(63924, 100)
(63924, 86)


In [15]:
print (x_train)

[[64 64 64 ... 16 32 64]
 [64 64 64 ... 76 34 37]
 [64 64 64 ... 64  7 73]
 ...
 [64 64 64 ... 64 32 49]
 [64 64 64 ... 73 57 64]
 [64 64 64 ... 41 54 16]]


In [16]:
import tensorflow as tf
import os
checkpoint_path = "cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [17]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=6,batch_size=1000,callbacks=[cp_callback])

Epoch 1/6
639/640 [============================>.] - ETA: 0s - loss: 2.2507 - accuracy: 0.3625
Epoch 00001: saving model to cp.ckpt
640/640 [==============================] - 79s 124ms/step - loss: 2.2507 - accuracy: 0.3625 - val_loss: 2.1836 - val_accuracy: 0.4250
Epoch 2/6
639/640 [============================>.] - ETA: 0s - loss: 1.8133 - accuracy: 0.4641
Epoch 00002: saving model to cp.ckpt
640/640 [==============================] - 81s 126ms/step - loss: 1.8133 - accuracy: 0.4641 - val_loss: 1.7964 - val_accuracy: 0.4737
Epoch 3/6
639/640 [============================>.] - ETA: 0s - loss: 1.6388 - accuracy: 0.5094
Epoch 00003: saving model to cp.ckpt
640/640 [==============================] - 80s 124ms/step - loss: 1.6388 - accuracy: 0.5094 - val_loss: 1.7458 - val_accuracy: 0.4860
Epoch 4/6
639/640 [============================>.] - ETA: 0s - loss: 1.5187 - accuracy: 0.5397
Epoch 00004: saving model to cp.ckpt
640/640 [==============================] - 81s 126ms/step - loss: 1.51

In [18]:
def predict(txt, word_gen):
    inp_len = len(txt) 
    temp_txt=str()
    if inp_len<100:
        
        for i in range(seq_length-len(txt)):
            temp_txt+=' '
        temp_txt += txt   
    else:
        temp_txt = txt[:100]

    int_txt=[char2idx[c] for c in temp_txt ]
    
    int_txt= np.array(int_txt)
    int_txt = np.reshape(int_txt,(1,100))

    
    for i in range(word_gen):
        
        pred = model.predict(int_txt)
        
        txt+=idx2char[np.argmax(pred)]
        int_txt[0][:99]=int_txt[0][1:]
        int_txt[0][99]=np.argmax(pred)

        
    return txt


In [19]:
print(predict('war means hate',400))

war means hate the still of the same to the same which he was
seen of the starter and the same was seemed this carnor and the same in
the saigh to began to the down and a shoute of the herse his arms all
hadked the subjectels of the from and himse went to the count of the
councious of the manighturin the seemed the fellows, but the front of
his faces and wite to reply defereft the his commander and a maving or 
